# Gauss Karışım Modelleri (GMM) kullanarak Beklenti-Maksimizasyon (EM) Kümeleme:

##### K-Ortalamalarının en büyük dezavantajlarından biri, küme merkezi için ortalama değerin naif kullanımıdır. Aşağıdaki resme bakarak bunun bir şeyleri yapmanın en iyi yolu olmadığını anlayabiliriz. Sol tarafta, aynı ortalamaya merkezlenmiş farklı yarıçaplara sahip iki dairesel küme olduğu insan gözüne oldukça açık görünüyor. K-Ortalamalar bunun üstesinden gelemez çünkü kümelerin ortalama değerleri birbirine çok yakındır. K-Ortalamalar, yine ortalamanın küme merkezi olarak kullanılması sonucunda kümelerin dairesel olmadığı durumlarda başarısız olur.

![Ekran görüntüsü 2024-05-17 072305](https://github.com/ahmettsimsek/Makine-Ogrenmesi-Temelleri/assets/124433579/7c808f65-f3bd-4ee9-a653-ddac58526da7)


#####  Gauss Karışım Modelleri (GMM'ler) bize K-Ortalamalarından daha fazla esneklik sağlar. GMM'ler ile veri noktalarının Gauss olarak dağıtıldığını varsayıyoruz; bu, ortalamayı kullanarak döngüsel olduklarını söylemekten daha az kısıtlayıcı bir varsayımdır. Bu şekilde, kümelerin şeklini açıklamak için iki parametremiz var: ortalama ve standart sapma! İki boyutlu bir örnek alırsak, bu, kümelerin her türlü eliptik şekli alabileceği anlamına gelir (çünkü hem x hem de y yönlerinde standart bir sapmaya sahibiz). Böylece, her Gauss dağılımı tek bir kümeye atanır.

##### Her küme için Gauss'un parametrelerini bulmak için (örneğin ortalama ve standart sapma), Beklenti-Maksimizasyon (EM) adı verilen bir optimizasyon algoritması kullanacağız. Kümelere uydurulan Gaussian'ların bir örneği olarak aşağıdaki grafiğe bir göz atın. Daha sonra GMM'leri kullanarak Beklenti-Maksimizasyon kümeleme sürecine geçebiliriz. Yani özetlememiz gerekirse, K-Means olmuyorsa Gauss ile yapıyoruz.

![Ekran görüntüsü 2024-05-17 072453](https://github.com/ahmettsimsek/Makine-Ogrenmesi-Temelleri/assets/124433579/bdcc0920-a6b9-4321-863d-edc33560016f)


##### Grafikte gördüğümüz gibi dairesel çizersek dışarıda kalırdı ancak Gauss ile daha elisik aldık ama bu durumda bile dışarda kalanlar olduğu için başarımızı düşürmektedir.

1) Küme sayısını seçerek (K-Means'ın yaptığı gibi) ve her küme için Gauss dağılım
parametrelerini rastgele başlatarak başlıyoruz. Verilere de hızlıca göz atarak ilk
parametreler için iyi bir tahmin sağlamaya çalışılabilir. Yine de, yukarıdaki grafikte de
görülebileceği gibi, bu% 100 gerekli değildir çünkü Gauss bize çok zayıf olarak
başlarlar, ancak hızla optimize edilirler.


2) Her küme için bu Gauss dağılımları göz önüne alındığında, her veri noktasının belirli
bir kümeye ait olma olasılığı hesaplanır.

3) Bu olasılıklara dayanarak, kümeler içindeki veri noktalarının olasılıklarını maksimize
edecek şekilde Gauss dağılımları için yeni bir dizi parametre hesaplıyoruz.  Ayrıca noktaların çoğunun “üstten-sağdan-aşağıya” olduğunu da görebiliriz. Bu nedenle standart sapma, olasılıkların ağırlıklandırdığı toplamı maksimize etmek için bu noktalara daha uygun bir elips oluşturmak üzere değişir.

4) Dağıtımların yinelemeden yinelemeye pek değişmediği yakınsamaya kadar 2. ve 3. adımlar yinelemeli olarak tekrarlanır.

##### GMM kullanmanın 2 önemli avantajı vardır. Birincisi,
GMM'ler küme kovaryansı açısından KOrtalamalarına göre çok daha esnektir; standart sapma parametresi nedeniyle, kümeler dairelerle sınırlı olmak yerine herhangi bir elips şeklini alabilir. Dolayısıyla, bir veri noktası üst üste binen iki kümenin ortasındaysa, sınıf 1'e yüzde X ve yüzde Y yüzde 2'ye ait olduğunu söyleyerek sınıfını tanımlayabiliriz. Yani GMM'ler karma üyeliği destekler.

----------

##### Örnek olarak alttaki veriye bakalım.

In [ ]:
data = np.loadtxt('biometric_data_simple.txt',delimiter=',')

women = data[data[:,0] == 1]
men = data[data[:,0] == 2]

plt.xlim(55,80)
plt.ylim(80,280)
plt.plot (women[:,1],women[:,2], 'b.')
plt.hold(True)
plt.plot (men[:,1],men[:,2], 'r.')
plt.xlabel('boy (inch)')
plt.ylabel('agirlik (pound)')
plt.savefig('mixnorm_1.png')

![Ekran görüntüsü 2024-05-17 072920](https://github.com/ahmettsimsek/Makine-Ogrenmesi-Temelleri/assets/124433579/0376843b-e461-4d89-b8db-37c296dfde71)


Bu grafik kadınlar ve erkeklerin boy (height) ve kilolarını (weight) içeren bir veri setinden geliyor, veri setinde erkekler ve kadınlara ait olan ölçümler önceden işaretlenmiş / etiketlenmiş (labeled), biz de bu işaretleri kullanarak kadınları kırmızı erkekleri mavi ile grafikledik. Ama bu işaretler / etiketler verilmiş olsun ya da olmasın, kavramsal olarak düşünürsek eğer bu veriye bir dağılım uydurmak (fit) istersek bir karışım kullanılması gerekli, çünkü iki tepe noktasiyle daha rahat temsil edileceğini düşündüğümüz bir durum var ortada.

In [ ]:
# Multivariate gaussian, contours
#
import scipy.stats
import em

data = np.loadtxt('biometric_data_simple.txt',delimiter=',')

women = data[data[:,0] == 1]
men = data[data[:,0] == 2]

plt.xlim(55,80)
plt.ylim(80,280)
plt.plot (women[:,1],women[:,2], 'b.')
plt.hold(True)
plt.plot (men[:,1],men[:,2], 'r.')
plt.xlabel('boy (inch)')
plt.ylabel('agirlik (pound)')
plt.hold(True)

x = np.arange(55., 80., 1)
y = np.arange(80., 280., 1)
X, Y = np.meshgrid(x, y)

Z = np.zeros(X.shape)
nx, ny = X.shape
mu1 = np.array([  72.89350086,  193.21741426])
sigma1 = np.matrix([[    7.84711283,    25.03111826],
                    [   25.03111826,  1339.70289046]])
for i in xrange(nx):
    for j in xrange(ny):
        Z[i,j] = em.norm_pdf(np.array([X[i,j], Y[i,j]]),mu1,sigma1)

levels = np.linspace(Z.min(), Z.max(), 4)

plt.contour(X, Y, Z, colors='b', levels=levels)
plt.hold(True)

Z = np.zeros(X.shape)
nx, ny = X.shape
mu2 = np.array([  66.15903841,  135.308125  ])
sigma2 = np.matrix([[  14.28189396,   51.48931033],
                    [  51.48931033,  403.09566456]])
for i in xrange(nx):
    for j in xrange(ny):
        Z[i,j] = em.norm_pdf(np.array([X[i,j], Y[i,j]]),mu2,sigma2)

levels = np.linspace(Z.min(), Z.max(), 4)

plt.contour(X, Y, Z, colors='r', levels=levels)
plt.savefig('mixnorm_2.png')

![Ekran görüntüsü 2024-05-17 073018](https://github.com/ahmettsimsek/Makine-Ogrenmesi-Temelleri/assets/124433579/b6dc09a9-0872-47ce-97fa-077e61dd84c4)


Bu karışım içindeki Gaussian'ları üstteki gibi çizebiliriz.

--------

Gauss Karışım Modelleri (GMM) kullanarak Beklenti-Maksimizasyon (EM) Kümeleme konusunda daha fazla detaya ulaşmak için, benim de örnek için Yararlandığım  sayfaya aşağıdaki linkten ulaşabilirsiniz:

https://burakbayramli.github.io/dersblog/stat/stat_110_gmm/gaussian_karisim_modeli__gmm__ile_kumelemek.html